<a href="https://colab.research.google.com/github/danhorsley/DS-Unit-4-Sprint-1-NLP/blob/master/kagwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

In [22]:
files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train (1).csv


{'sample_submission.csv': b'id,category\n955,1\n3532,3\n1390,2\n1024,4\n1902,2\n1156,1\n2205,3\n885,4\n1116,3\n3739,4\n4003,4\n152,2\n930,2\n2840,3\n2902,4\n2622,2\n3158,4\n1290,3\n1731,2\n1537,1\n1897,4\n752,1\n343,1\n4004,2\n3021,1\n2162,1\n112,2\n2538,2\n3862,1\n2498,3\n3492,2\n850,2\n1158,1\n1505,4\n1229,1\n986,4\n3215,3\n4032,2\n706,1\n2459,3\n1336,2\n1836,3\n3381,1\n2794,1\n3115,4\n531,2\n921,2\n693,1\n1929,4\n2412,1\n3086,1\n1185,1\n2690,1\n2306,2\n3390,1\n1616,4\n498,2\n439,1\n2202,4\n855,4\n3693,1\n357,4\n1200,2\n462,4\n267,2\n720,2\n2139,4\n2093,1\n2255,1\n1355,2\n3141,3\n3435,3\n1637,2\n2702,2\n2309,4\n2886,2\n3287,2\n668,1\n2878,3\n2836,3\n494,3\n423,2\n485,1\n1526,3\n339,3\n4068,1\n332,4\n3989,4\n578,2\n2438,1\n3395,2\n2211,2\n366,4\n1928,2\n1903,3\n3265,3\n1953,2\n2865,1\n2003,3\n810,1\n2269,3\n2150,4\n557,4\n572,2\n3746,2\n470,2\n1499,4\n707,4\n2503,2\n636,2\n2787,2\n3920,3\n620,1\n1472,1\n2181,3\n3819,3\n3745,4\n3412,1\n1387,1\n1509,2\n2670,4\n2599,3\n1448,4\n1877,4\n28

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
train = pd.read_csv('train.csv',index_col='id')
test = pd.read_csv('test.csv',index_col='id')

In [269]:
train.head()

,author,description,price,ratingValue,pert_alcohol,category
id,,,,,,
1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.5,2.0
2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.9,1.0
3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.0,2.0
4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.5,1.0
6,Davin de Kergommeaux,"After 40 years in barrels, the trademark Canad...",199.0,96,45.0,NaN


In [0]:
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)

def tokenmate(frame):
  tokens = []

  for doc in tokenizer.pipe(frame['description'], batch_size=500):

      doc_tokens = []

      for token in doc: 
        if (token.is_stop == False) and (token.is_punct==False) and (token.is_space==False) :
          doc_tokens.append(token.text.lower())

      tokens.append(doc_tokens)

  frame['tokens'] = tokens

In [271]:
tokenmate(train)
tokenmate(test)
test.head()

,author,description,price,ratingValue,pert_alcohol,tokens
id,,,,,,
955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0,"[think, carnival, aromas—the, good, ones,, any..."
3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3,"[blend, bourbons,, 6, 12, years, old;, rye, wh..."
1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0,"[nose, focused, cereal,, hints, fresh, ripe, c..."
1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8,"[swiss-based, chapter, 7, released, 19, year, ..."
1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9,"[valkyrie, replaces, current, dark, origins, e..."


In [0]:
import category_encoders as ce
oe = ce.ordinal.OrdinalEncoder()
oe.fit(train[['author']])
train['author'] = oe.transform(train['author'])
test['author'] = oe.transform(test['author'])

In [0]:
nonverbals = ['price',	'ratingValue'	,'pert_alcohol']
ss.fit(train[nonverbals])
train[nonverbals]=ss.transform(train[nonverbals])
test[nonverbals]=ss.transform(test[nonverbals])

In [0]:
test=test.reset_index()

In [275]:
train.head(10)

,author,description,price,ratingValue,pert_alcohol,category,tokens
id,,,,,,,
1,1,A marriage of 13 and 18 year old bourbons. A m...,-0.141652,2.358463,0.548953,2.0,"[marriage, 13, 18, year, old, bourbons., matur..."
2,2,There have been some legendary Bowmores from t...,13.402787,2.358463,-0.816688,1.0,"[legendary, bowmores, mid-60s, bit, equal., sh..."
3,1,This bottling celebrates master distiller Park...,-0.076024,2.358463,0.310760,2.0,"[bottling, celebrates, master, distiller, park..."
4,1,What impresses me most is how this whisky evol...,4.315948,2.358463,-1.197797,1.0,"[impresses, whisky, evolves;, it's, incredibly..."
6,3,"After 40 years in barrels, the trademark Canad...",-0.026552,2.136772,-0.483218,NaN,"[40, years, barrels,, trademark, canadian, clu..."
9,4,"A caramel-laden fruit bouquet, followed by une...",-0.076024,2.136772,1.023752,2.0,"[caramel-laden, fruit, bouquet,, followed, une..."
12,1,Distilled at the now legendary Stitzel-Weller ...,0.075423,2.136772,-0.483218,2.0,"[distilled, legendary, stitzel-weller, distill..."
14,1,"No age statement, but distilled in 1998. A bea...",-0.146700,2.136772,3.335814,2.0,"[age, statement,, distilled, 1998., beautiful,..."
15,3,"Monarch, the 75th anniversary limited edition ...",-0.151748,2.136772,-1.277195,4.0,"[monarch,, 75th, anniversary, limited, edition..."


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 15)

def add_vectors(frame):
  feats = ['tokens','author']
  for feat in feats:
    mystrings = [' '.join(x) for x in list(train['tokens'])]
    tfidf.fit(mystrings)
    dtm = tfidf.transform([' '.join(x) for x in list(frame['tokens'])])
    docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
    #docs.head()
  return pd.merge(frame,docs,left_index=True,right_index=True)

In [0]:
trainvects = add_vectors(train)
testvects = add_vectors(test)

In [223]:
trainvects.shape,trainvects.isna().sum()

((2009, 17), author            0
 description       0
 price            59
 ratingValue       0
 pert_alcohol     42
 category        216
 tokens            0
 finish            0
 fruit             0
 nose              0
 notes             0
 oak               0
 old               0
 palate            0
 sweet             0
 vanilla           0
 whisky            0
 dtype: int64)

In [0]:
trainvects = trainvects[~trainvects['category'].isna()]


In [280]:
trainvects.columns

Index(['author', 'description', 'price', 'ratingValue', 'pert_alcohol',
       'category', 'tokens', 'caramel', 'finish', 'fruit', 'it', 'nose',
       'notes', 'oak', 'old', 'palate', 'sherry', 'spice', 'sweet', 'vanilla',
       'whisky', 'year'],
      dtype='object')

In [0]:
features = ['author',  'price', 'ratingValue', 'pert_alcohol',
        'caramel', 'finish', 'fruit', 'it', 'nose',
       'notes', 'oak', 'old', 'palate', 'sherry', 'spice', 'sweet', 'vanilla',
       'whisky', 'year']

target = ['category']

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [0]:
from sklearn.model_selection import cross_validate
trainvects['pert_alcohol']=trainvects['pert_alcohol'].fillna(trainvects['pert_alcohol'].mean())
trainvects['price']=trainvects['price'].fillna(trainvects['price'].mean())
testvects['pert_alcohol']=testvects['pert_alcohol'].fillna(testvects['pert_alcohol'].mean())
testvects['price']=testvects['price'].fillna(testvects['price'].mean())


In [0]:
ss = StandardScaler()
rfc = RandomForestClassifier(n_estimators=150,max_depth=15)
sgd = SGDClassifier()
xgb = XGBClassifier(n_estimators=150,max_depth=4)
lr = LogisticRegression(solver='lbfgs')
models = [rfc,sgd,xgb,lr]

In [0]:
X = trainvects[features]
y = trainvects[target]

In [286]:
#ss.fit_transform(X)
X.head()

,author,price,ratingValue,pert_alcohol,caramel,finish,fruit,it,nose,notes,oak,old,palate,sherry,spice,sweet,vanilla,whisky,year
1,1,-0.141652,2.358463,0.548953,0.0,0.000000,0.777930,0.000000,0.000000,0.000000,0.000000,0.000000,0.628351,0.0,0.0,0.000000,0.00000,0.000000,0.000000
2,2,13.402787,2.358463,-0.816688,0.0,0.171803,0.000000,0.306355,0.000000,0.000000,0.227007,0.578401,0.197104,0.0,0.0,0.000000,0.22628,0.000000,0.632849
3,1,-0.076024,2.358463,0.310760,0.0,0.000000,0.227683,0.285840,0.201161,0.208832,0.211805,0.000000,0.367809,0.0,0.0,0.000000,0.00000,0.776038,0.000000
4,1,4.315948,2.358463,-1.197797,0.0,0.388766,0.552191,0.000000,0.000000,0.000000,0.513684,0.000000,0.000000,0.0,0.0,0.529221,0.00000,0.000000,0.000000
9,4,-0.076024,2.136772,1.023752,0.0,0.339030,0.000000,0.000000,0.000000,0.441679,0.000000,0.570697,0.388957,0.0,0.0,0.461516,0.00000,0.000000,0.000000


In [287]:
for model in models :
  cv_results = cross_validate(model, X,y,scoring ='accuracy', cv=5)
  print(cv_results)
  print(cv_results['test_score'].mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 

{'fit_time': array([0.44318986, 0.43546772, 0.43479276, 0.44364405, 0.43632531]), 'score_time': array([0.01653481, 0.01772499, 0.01728821, 0.01733708, 0.01648211]), 'test_score': array([0.41274238, 0.83055556, 0.83798883, 0.86834734, 0.77030812])}
0.743988445365508
{'fit_time': array([0.03769898, 0.02294612, 0.03323913, 0.02879763, 0.02205682]), 'score_time': array([0.00185418, 0.00157762, 0.00162172, 0.00161886, 0.00163555]), 'test_score': array([0.69252078, 0.81944444, 0.68715084, 0.74229692, 0.72268908])}
0.7328204104908599


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

{'fit_time': array([1.02801657, 1.05063772, 1.03472447, 1.04756618, 1.03661919]), 'score_time': array([0.01091719, 0.01096582, 0.01189852, 0.01147914, 0.01287484]), 'test_score': array([0.61772853, 0.88888889, 0.89106145, 0.92717087, 0.82633053])}
0.830236054763807


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr

{'fit_time': array([0.10795832, 0.12999201, 0.1094594 , 0.11511135, 0.11293864]), 'score_time': array([0.00192642, 0.00188661, 0.00188661, 0.00186515, 0.00188923]), 'test_score': array([0.66759003, 0.85555556, 0.74301676, 0.73669468, 0.74509804])}
0.7495910120239515


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
Xscaled = StandardScaler().fit_transform(X)

In [195]:
#best model looks like SGDClassifier
from sklearn.model_selection import RandomizedSearchCV



param_distributions = {
    'alpha': [0.01,0.001,0.0001], 
    'average': [True,False], 
    'early_stopping': [True,False],
    'epsilon' :[0.05,0.1,0.2],
    'eta0' :[0,0.01,0.1],
    'fit_intercept' :[True,False]
}

gridsearch = RandomizedSearchCV(
    SGDClassifier(n_jobs=-1, random_state=55, class_weight = {1:0.633,2:0.173,3:0.116,4:0.078}), 
    param_distributions=param_distributions, 
    n_iter=30, 
    cv=5, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True, 
    n_jobs=-1
)

gridsearch.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   13.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight={1: 0.633, 2: 0.173,
                                                         3: 0.116, 4: 0.078},
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=-1,
                                           penalty='l2', power_t=0.5,
                                           random_state=55,...
                                           tol=0.001, validation_fraction=0.1,
                                           verbose=0, warm_start=False),
       

In [196]:
gridsearch.best_estimator_

SGDClassifier(alpha=0.0001, average=True,
              class_weight={1: 0.633, 2: 0.173, 3: 0.116, 4: 0.078},
              early_stopping=False, epsilon=0.2, eta0=0.1, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=55, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [197]:
results = pd.DataFrame(gridsearch.cv_results_)
print(f'Best result from search of {len(results)} parameter combinations')
results.sort_values(by='rank_test_score').head(3)

Best result from search of 30 parameter combinations


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fit_intercept,param_eta0,param_epsilon,param_early_stopping,param_average,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
29,0.144985,0.038990,0.002544,0.000349,False,0,0.05,False,True,0.0001,"{'fit_intercept': False, 'eta0': 0, 'epsilon':...",0.725762,0.719444,0.712291,0.756303,0.697479,0.722253,0.019414,1,0.74581,0.702024,0.721951,0.729109,0.729109,0.7256,0.014158
5,0.145023,0.048255,0.002250,0.000135,False,0.1,0.2,False,True,0.0001,"{'fit_intercept': False, 'eta0': 0.1, 'epsilon...",0.725762,0.719444,0.712291,0.756303,0.697479,0.722253,0.019414,1,0.74581,0.702024,0.721951,0.729109,0.729109,0.7256,0.014158
9,0.149613,0.038839,0.002348,0.000330,False,0,0.2,False,True,0.0001,"{'fit_intercept': False, 'eta0': 0, 'epsilon':...",0.725762,0.719444,0.712291,0.756303,0.697479,0.722253,0.019414,1,0.74581,0.702024,0.721951,0.729109,0.729109,0.7256,0.014158


In [200]:
train['category'].value_counts(normalize=True)

1.0    0.633024
2.0    0.173627
3.0    0.116009
4.0    0.077340
Name: category, dtype: float64

In [0]:
bc = SGDClassifier(alpha=0.0001, average=True,
              class_weight={1: 0.633, 2: 0.173, 3: 0.116, 4: 0.078},
              early_stopping=False, epsilon=0.2, eta0=0.1, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=55, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [199]:
bc.fit(X,y)
my_preds = bc.predict(testvects[features])
my_preds = [int(x) for x in my_preds]

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
testvects[features].isna().sum()

In [153]:
samp = pd.read_csv('sample_submission.csv',index_col='id')
samp['category']=my_preds
samp.head()

,category
id,
955,2
3532,1
1390,1
1024,1
1902,1


In [0]:
samp.to_csv('mysub3')

In [0]:
files.download('mysub3')

In [150]:
samp['category'].value_counts(normalize=True)

1.0    0.854167
2.0    0.145833
Name: category, dtype: float64

In [234]:
from sklearn.model_selection import RandomizedSearchCV



param_distributions = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

}

gridsearch = RandomizedSearchCV(
    LogisticRegression(n_jobs=-1, random_state=55), 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=5, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True, 
    n_jobs=-1
)

gridsearch.fit(X, y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   12.2s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='warn', n_jobs=-1,
                                                penalty='l2', random_state=55,
                                                solver='warn', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True

In [235]:
results = pd.DataFrame(gridsearch.cv_results_)
print(f'Best result from search of {len(results)} parameter combinations')
results.sort_values(by='rank_test_score').head(3)

Best result from search of 5 parameter combinations


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,2.667586,1.182267,0.002384,0.000165,newton-cg,{'solver': 'newton-cg'},0.706371,0.775000,0.768156,0.717087,0.672269,0.727830,0.038753,1,0.749302,0.749477,0.745645,0.752089,0.752786,0.749860,0.002520
1,0.822256,0.268582,0.002166,0.000158,lbfgs,{'solver': 'lbfgs'},0.745152,0.772222,0.745810,0.700280,0.672269,0.727273,0.035862,2,0.746508,0.745290,0.737979,0.738858,0.748607,0.743448,0.004251
2,0.068224,0.017581,0.002124,0.000053,liblinear,{'solver': 'liblinear'},0.725762,0.758333,0.703911,0.717087,0.666667,0.714445,0.029843,3,0.709497,0.730635,0.730314,0.713092,0.728412,0.722390,0.009162


In [304]:

from sklearn.model_selection import RandomizedSearchCV



param_distributions = {
    'booster': ['gbtree','gblinear'],
    'max_depth': [3,5],
    'learning_rate': [0.001,0.01,0.05,0.1,0.2],
    'n_estimators': [125,150,175,200,250]

}

gridsearch = RandomizedSearchCV(
    XGBClassifier(n_jobs=-1, random_state=100,class_weight=cw), 
    param_distributions=param_distributions, 
    n_iter=30, 
    cv=5, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True, 
    n_jobs=-1
)

gridsearch.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel()

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           class_weight={1.0: 0.6330239752513535,
                                                         2.0: 0.17362722351121423,
                                                         3.0: 0.11600928074245939,
                                                         4.0: 0.07733952049497293},
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weigh...
                                           reg_lambda=1, scale_pos_weight=1,
                                           seed=None, silent=None, subsample=1,
                      

In [306]:
results = pd.DataFrame(gridsearch.cv_results_)
print(f'Best result from search of {len(results)} parameter combinations')
results.sort_values(by='rank_test_score').head(5)

Best result from search of 30 parameter combinations


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,param_booster,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
6,1.900963,0.013324,0.018656,0.002243,250,3,0.05,gbtree,"{'n_estimators': 250, 'max_depth': 3, 'learnin...",0.631579,0.897222,0.907821,0.929972,0.831933,0.839375,0.109299,1,0.983240,0.981158,0.979791,0.980501,0.986072,0.982153,0.002273
26,2.287733,0.035165,0.021531,0.000604,250,5,0.2,gbtree,"{'n_estimators': 250, 'max_depth': 5, 'learnin...",0.664820,0.880556,0.871508,0.913165,0.837535,0.833240,0.087912,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,1.921372,0.027672,0.017282,0.000722,175,5,0.1,gbtree,"{'n_estimators': 175, 'max_depth': 5, 'learnin...",0.606648,0.891667,0.879888,0.927171,0.837535,0.828221,0.114859,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,2.594790,0.032655,0.023371,0.001485,250,5,0.1,gbtree,"{'n_estimators': 250, 'max_depth': 5, 'learnin...",0.609418,0.886111,0.874302,0.924370,0.848739,0.828221,0.112518,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
24,2.274417,0.034570,0.017458,0.000545,200,5,0.05,gbtree,"{'n_estimators': 200, 'max_depth': 5, 'learnin...",0.576177,0.894444,0.893855,0.927171,0.817927,0.821528,0.128284,5,0.996508,0.997209,0.997213,0.995822,0.995822,0.996515,0.000621


In [0]:
myxgb = gridsearch.best_estimator_

In [311]:
myxgb.fit(X,y)
my_preds = myxgb.predict(testvects[features])
my_preds = [int(x) for x in my_preds]
samp = pd.read_csv('sample_submission.csv',index_col='id')
samp['category']=my_preds
samp.to_csv('mysub4.csv')
files.download('mysub4.csv')

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [305]:
cw=dict(train['category'].value_counts(normalize=True))
cw

{1.0: 0.6330239752513535,
 2.0: 0.17362722351121423,
 3.0: 0.11600928074245939,
 4.0: 0.07733952049497293}